<a href="https://colab.research.google.com/github/dookda/cmu_udfire_gee/blob/main/predict_hp_using_lstm_gee_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip -q install -U geemap

!pip -q install -U geemap earthengine-api pandas numpy scikit-learn tensorflow geopandas shapely rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.5/631.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.6 MB/s eta 0:00:00


In [7]:
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import ee, geemap, time
from datetime import datetime

ee.Authenticate()
try:
    ee.Initialize(project="ee-sakda-451407")
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project="ee-sakda-451407")

In [17]:
# --- AOI จากผู้ใช้ ---
aoi = ee.Geometry.Polygon(
    [[[98.42352973176759, 18.531322241927942],
      [98.42352973176759, 18.44601436309291],
      [98.52446662141602, 18.44601436309291],
      [98.52446662141602, 18.531322241927942]]]
)

# --- Parameters ---
START_DATE = '2018-01-01'
END_DATE   = '2024-12-31'
CRS_EPSG   = 'EPSG:32647'   # UTM zone 47N (เหมาะกับเชียงใหม่)
GRID_SIZE_M = 1000
N_ROWS, N_COLS = 10, 10

# --- Build 1km grid (10x10) ---
def make_grid_1km(aoi, crs_epsg, n_rows, n_cols, cell_m=1000):
    proj = ee.Projection(crs_epsg)
    aoi_proj = aoi.transform(proj, 1)
    bounds = aoi_proj.bounds(1, proj)
    coords = ee.List(bounds.coordinates().get(0))
    xs = coords.map(lambda p: ee.Number(ee.List(p).get(0)))
    ys = coords.map(lambda p: ee.Number(ee.List(p).get(1)))
    minx = ee.Number(ee.List(xs).reduce(ee.Reducer.min()))
    miny = ee.Number(ee.List(ys).reduce(ee.Reducer.min()))

    # snap ลงกริด 1,000 ม.
    def snap(v, base):
        v = ee.Number(v); base = ee.Number(base)
        return v.divide(base).floor().multiply(base)
    minx = snap(minx, cell_m)
    miny = snap(miny, cell_m)

    def mk_cell(i, j):
        i = ee.Number(i); j = ee.Number(j); cm = ee.Number(cell_m)
        x0 = minx.add(i.multiply(cm))
        y0 = miny.add(j.multiply(cm))
        x1 = minx.add(i.add(1).multiply(cm))    # แก้จาก i+1 -> i.add(1)
        y1 = miny.add(j.add(1).multiply(cm))    # แก้จาก j+1 -> j.add(1)
        rect = ee.Geometry.Rectangle([x0, y0, x1, y1], proj, False)
        return ee.Feature(rect, {
            'row': i,
            'col': j,
            'cell_id': i.multiply(n_rows).add(j)
        })

    cells = ee.FeatureCollection(
        ee.List.sequence(0, n_cols - 1).map(
            lambda i: ee.List.sequence(0, n_rows - 1).map(
                lambda j: mk_cell(i, j)
            )
        ).flatten()
    ).filterBounds(aoi_proj)

    return cells

grid_fc = make_grid_1km(aoi, CRS_EPSG, N_ROWS, N_COLS, GRID_SIZE_M)

# --- MODIS collections ---
def get_modis_vi():
    ic = ee.ImageCollection('MODIS/061/MOD13A2').select(['NDVI','EVI'])
    # scale factor 0.0001
    return ic.map(lambda img: img.multiply(0.0001).copyProperties(img, img.propertyNames()))

def get_modis_lst_day():
    ic = ee.ImageCollection('MODIS/061/MOD11A2').select(['LST_Day_1km'])
    # scale 0.02 K -> °C
    def to_c(img):
        return img.multiply(0.02).subtract(273.15).rename('LST_Day_C') \
                  .copyProperties(img, img.propertyNames())
    return ic.map(to_c)

vi_ic    = get_modis_vi()
lst_ic   = get_modis_lst_day()
firms_ic = ee.ImageCollection('FIRMS')

# --- Weekly range ---
START, END = ee.Date(START_DATE), ee.Date(END_DATE)
n_weeks = END.difference(START, 'week').int()
week_starts = ee.List.sequence(0, n_weeks.subtract(1)).map(lambda k: START.advance(ee.Number(k), 'week'))

proj = ee.Projection(CRS_EPSG).atScale(GRID_SIZE_M)

def reduce_one_week(ws):
    ws = ee.Date(ws)
    we = ws.advance(1, 'week')

    ndvi_evi = vi_ic.filterDate(ws, we).mean()          # NDVI,EVI (scaled)
    lst      = lst_ic.filterDate(ws, we).mean()         # °C

    # FIRMS: ใช้ select([0]) เพื่อความทั่วไป (บางคอลเลกชันชื่อแบนด์ต่างกัน)
    fire_cnt = firms_ic.filterDate(ws, we).select([0]).count().rename('FIRE_COUNT')

    stack = ndvi_evi.addBands(lst).addBands(fire_cnt).reproject(proj)

    reduced = stack.reduceRegions(
        collection = grid_fc,
        reducer    = ee.Reducer.mean(),
        scale      = GRID_SIZE_M
    ).map(lambda f: f.set({
        'week_start': ws.format('YYYY-MM-dd'),
        'label': ee.Number(f.get('FIRE_COUNT')).gt(0).int()
    }))
    return reduced

weekly_fc = ee.FeatureCollection(week_starts.map(reduce_one_week)).flatten() \
            .map(lambda f: f.setGeometry(None))  # ตารางล้วน

EXPORT_FOLDER = "_GEE_TUM"
# --- Export tasks ---
task1 = ee.batch.Export.table.toDrive(
    collection = weekly_fc,
    description = 'weekly_modis_firms_cm_1km_10x10',
    fileFormat = 'CSV',
    folder = EXPORT_FOLDER
)
task1.start()

task2 = ee.batch.Export.table.toDrive(
    collection = grid_fc,
    description = 'grid_cm_1km_10x10_geojson',
    fileFormat = 'GeoJSON',
    folder = EXPORT_FOLDER
)
task2.start()

print("Export tasks started.")

# (ตัวเลือก) เฝ้าดูสถานะงานส่งออก
def wait_for(tasks, poll_sec=30):
    while True:
        states = [t.status().get('state') for t in tasks]
        print(datetime.now(), states)
        if all(s in ('COMPLETED','FAILED','CANCELLED') for s in states):
            break
        time.sleep(poll_sec)

# เปิดใช้งานถ้าต้องการรอจนเสร็จใน Colab
# wait_for([task1, task2], poll_sec=60)


Export tasks started.


In [12]:
# pip install pandas numpy scikit-learn tensorflow geopandas shapely rasterio
import pandas as pd, numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, average_precision_score
import tensorflow as tf

# 1) โหลดข้อมูล
df = pd.read_csv('weekly_modis_fires_1km_10x10.csv')
# คาดว่ามีคอลัมน์: row, col, cell_id, week_start, NDVI, EVI, LST_Day_C, FIRE_COUNT_mean (หรือ sum), label

# 2) เตรียมเวลา/คีย์
df['week_start'] = pd.to_datetime(df['week_start'])
df = df.sort_values(['cell_id','week_start'])

# 3) เลือกฟีเจอร์และสร้างล่าช้า (optional สำหรับ ANN; LSTM ไม่จำเป็นต้อง explicit lag)
features = ['NDVI','EVI','LST_Day_C','FIRE_COUNT']  # ชื่อให้ตรงกับที่ export
df = df.dropna(subset=features + ['label'])

# 4) สร้างลำดับเวลา per-cell
SEQ_LEN = 8   # ใช้ข้อมูล 8 สัปดาห์ย้อนหลังเพื่อทำนายสัปดาห์ถัดไป
X_list, y_list, meta = [], [], []
for cid, g in df.groupby('cell_id'):
    g = g.sort_values('week_start')
    vals = g[features].values
    ys   = g['label'].values
    for i in range(len(g) - SEQ_LEN):
        X_list.append(vals[i:i+SEQ_LEN])
        y_list.append(ys[i+SEQ_LEN])  # ทำนายสัปดาห์ถัดไป
        meta.append((cid, g.iloc[i+SEQ_LEN]['week_start']))

X = np.array(X_list)  # (N, SEQ_LEN, n_feat)
y = np.array(y_list).astype(np.float32)

# 5) แบ่ง train/val/test แบบ time-based (ตัวอย่าง: 70/15/15)
N = len(y)
train_end = int(N*0.7)
val_end   = int(N*0.85)
X_train, y_train = X[:train_end], y[:train_end]
X_val,   y_val   = X[train_end:val_end], y[train_end:val_end]
X_test,  y_test  = X[val_end:], y[val_end:]

# สเกลฟีเจอร์: fit จาก train เท่านั้น
scaler = StandardScaler()
X_train_2d = X_train.reshape(-1, X.shape[-1])
scaler.fit(X_train_2d)
def apply_scale(Xin):
    s = Xin.reshape(-1, Xin.shape[-1])
    s = scaler.transform(s)
    return s.reshape(Xin.shape)

X_train = apply_scale(X_train)
X_val   = apply_scale(X_val)
X_test  = apply_scale(X_test)

# 6) โมเดล LSTM
tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(SEQ_LEN, X.shape[-1])),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve='PR', name='pr_auc'),
                       tf.keras.metrics.AUC(curve='ROC', name='roc_auc')])

# จัด class weights แก้ imbalance (ถ้า hotspot = 1 น้อย)
pos = (y_train==1).sum()
neg = (y_train==0).sum()
cw = {0: 1.0, 1: max(1.0, neg/max(1,pos))}

model.fit(X_train, y_train, epochs=20, batch_size=128,
          validation_data=(X_val, y_val), class_weight=cw,
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)])

# 7) ประเมิน
proba = model.predict(X_test).ravel()
print('PR-AUC:', average_precision_score(y_test, proba))
print(classification_report(y_test, (proba>=0.5).astype(int)))


AttributeError: module 'geemap' has no attribute 'ee_to_pandas'